# Preprocessing PDF


In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [2]:

local_path = "PDF/JPTK-1.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [3]:
# Preview first page
text = data[0].page_content
print(text)

ISSN0216-3241 1

STUDI KELAYAKAN PEMBUKAAN PROGRAM DIPLOMA III TEKNOLOGI INFORMASI UNDIKSHA SINGARAJA

Oleh: Komang Setemen, Putu Hendra Suputra, Ketut Purnamawan Jurusan Manajemen Informatika, FTK, Undiksha

Abstrak

Bali sebagai daerah tujuan wisata (DTW) yang makin mengglobal memberikan sejumlah peluang dan tantangan, seperti : terbukanya sejumlah lapangan kerja baru, baik dalam jenis, jumlah, dan jenjang kualifikasi tertentu. Lapangan pekerjaan itu sebagian besar bertalian langsung dengan industri pariwisata serta sistem pendukungnya. Misalnya, jasa bahasa asing (Inggris dll), manajemen bisnis dan akuntansi, pemandu wisata, biro perjalanan, pengelolaan hotel, sport dan rekreasi, kesenian, produk industri kecil/kerajinan, termasuk teknologi informasi. Secara umum maupun secara khusus di Bali, sampai saat ini bidang teknologi informasi sudah memasuki hampir seluruh cabang kehidupan dengan kecenderungan yang semakin meningkat. Untuk mengetahui kompetensi dan potensi kebutuhan lulusan 

# Vektorisasi


In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [5]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

print(chunks)

[Document(metadata={'source': 'PDF/JPTK-1.pdf'}, page_content='ISSN0216-3241 1\n\nSTUDI KELAYAKAN PEMBUKAAN PROGRAM DIPLOMA III TEKNOLOGI INFORMASI UNDIKSHA SINGARAJA\n\nOleh: Komang Setemen, Putu Hendra Suputra, Ketut Purnamawan Jurusan Manajemen Informatika, FTK, Undiksha\n\nAbstrak\n\nBali sebagai daerah tujuan wisata (DTW) yang makin mengglobal memberikan sejumlah peluang dan tantangan, seperti : terbukanya sejumlah lapangan kerja baru, baik dalam jenis, jumlah, dan jenjang kualifikasi tertentu. Lapangan pekerjaan itu sebagian besar bertalian langsung dengan industri pariwisata serta sistem pendukungnya. Misalnya, jasa bahasa asing (Inggris dll), manajemen bisnis dan akuntansi, pemandu wisata, biro perjalanan, pengelolaan hotel, sport dan rekreasi, kesenian, produk industri kecil/kerajinan, termasuk teknologi informasi. Secara umum maupun secara khusus di Bali, sampai saat ini bidang teknologi informasi sudah memasuki hampir seluruh cabang kehidupan dengan kecenderungan yang semaki

In [6]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 5/5 [02:20<00:00, 28.08s/it]


# Retrival


In [7]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [8]:
# LLM from Ollama
local_model = "gemma2"
llm = ChatOllama(model=local_model)

In [9]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""(Gunakan bahasa indonesia) Anda adalah model AI untuk menghasilkan teks berdasarkan database vektor. Berikan jawaban hanya berdasarkan database dokumen yang saya berikan di database vektor. Jika kamu tidak menemukan jawaban, berikan jawaban bahwa kamu tidak mempelajarinya, Buatkan jawaban dengan versi yang berbeda untuk setiap pertanyaan yang mirip atau bahkan sama. Selain itu untuk pertanyaan yang pertama gunakan salam pembuka "Salam Harmoni".
    Original question: {question}""",
)

In [10]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
chain.invoke()

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [15]:
import os
from crewai import Agent, Task, Crew
from langchain_community.chat_models import ChatOllama

def write_blog(topic, chain):
  # Create Researcher agent
  researcher = Agent(
      role='Analis Jurnal',
      goal=f"Menemukan jurnal penelitian tentang {topic}",
      backstory="""Anda bekerja di perusahaan riset terkemuka.
      Keahlian Anda terletak pada mengidentifikasi tren yang sedang berkembang, melakukan riset, dan menemukan informasi baru.
      Anda ahli dalam menganalisis data yang kompleks dan menyajikan wawasan yang dapat ditindaklanjuti.""",
      verbose=True,
      allow_delegation=False,
      llm=chain
  )

  # Create Writer agent
  writer = Agent(
      role='Kreator Konten',
      goal=f"Membuat konten yang menarik tentang {topic}",
      backstory="""Anda adalah seorang Kreator Konten terkenal, dikenal dengan artikel-artikel Anda yang informatif dan menarik.
      Anda mengubah konsep yang kompleks menjadi narasi yang menarik.""",
      verbose=True,
      allow_delegation=False,
      llm=chain
  )

  # Create tasks for your agents
  task1 = Task(
    description=f"Lakukan analisis komprehensif tentang {topic}",
    expected_output="Laporan analisis dalam bentuk bullet poin",
    agent=researcher
  )

  task2 = Task(
    description=f"""Dengan menggunakan wawasan yang diberikan, buatlah blog post yang menarik
    yang menyoroti topik {topic}.
    Tulisan Anda harus informatif namun mudah dipahami, cocok untuk semua audiens, mudah dimengerti.
    Buat terdengar keren, hindari kata-kata kompleks agar tidak terdengar seperti AI..""",
    expected_output="Blog post lengkap setidaknya 4 paragraf menggunakan bahasa indonesia",
    agent=writer
  )

  # Instantiate your crew with a sequential process
  crew = Crew(
      agents=[researcher, writer],
      tasks=[task1, task2],
      verbose=True, 
  )

  # Get your crew to work!
  result = crew.kickoff()
  return result

response = write_blog('siapa penulisnya', llm)
print(response)

2024-08-21 11:25:01,236 - 19672 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-21 11:25:01][DEBUG]: == Working Agent: Analis Riset Senior
 [2024-08-21 11:25:01][INFO]: == Starting Task: Lakukan analisis komprehensif tentang siapa penulisnya


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer:  Please provide me with the text or material you want me to analyze. I need the content to determine the author. 

For example, you could say:

"Do an analysis of who wrote this article: [Insert article text here]" 


Once I have the text, I can perform a comprehensive analysis and give you a report in bullet points identifying potential authors based on:

* **Writing style:**  I'll look for patterns in sentence structure, vocabulary choice, tone, and overall voice.
* **Content expertise:** The subject matter discussed can offer clues about the author's field of knowledge or specialization.
* **Citations and references:**  Authors often cite their own work or that of colleagues, which can lead to identifying them.
* **Pub

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL halaman web yang ingin di-scrape
url = "https://ejournal.undiksha.ac.id/index.php/JPI/article/view/1414/1275"

# Mengirimkan permintaan HTTP ke URL
response = requests.get(url)

# Memeriksa apakah permintaan berhasil
if response.status_code == 200:
    # Parsing konten HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Menyimpan kode HTML ke dalam variabel
    html_content = soup.prettify()

    # Menyimpan HTML ke file .txt
    with open("output.txt", "w", encoding="utf-8") as file:
        file.write(html_content)
    print("HTML berhasil disimpan ke file 'output.txt'")
else:
    print(f"Permintaan gagal dengan status code {response.status_code}")


ConnectionError: HTTPSConnectionPool(host='ejournal.undiksha.ac.id', port=443): Max retries exceeded with url: /index.php/JPI/article/view/1414/1275 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000201AADCA850>: Failed to resolve 'ejournal.undiksha.ac.id' ([Errno 11001] getaddrinfo failed)"))